# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

I choose the Fashion Trends Data, as it provides the relevant context for RAG approach. 

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
import pandas as pd
import numpy as np
import openai
import tiktoken

df = pd.read_csv('2023_fashion_trends.csv')

df['Trends']

0     2023 Fashion Trend: Red. Glossy red hues took ...
1     2023 Fashion Trend: Cargo Pants. Utilitarian w...
2     2023 Fashion Trend: Sheer Clothing. "Bare it a...
3     2023 Fashion Trend: Denim Reimagined. From dou...
4     2023 Fashion Trend: Shine For The Daytime. The...
                            ...                        
77    If lime green isn't your vibe, rest assured th...
78    "As someone who can clearly (not fondly) remem...
79    "Combine this design shift with the fact that ...
80    Thought party season ended at the stroke of mi...
81    "This season, we saw the revival of the bubble...
Name: Trends, Length: 82, dtype: object

In [2]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
response = openai.embeddings.create(input = df["Trends"], model=EMBEDDING_MODEL_NAME)

In [3]:
#Add Embeddings and Token Count to DataFrame
df['embeddings'] = [None] * len(df.index)
df['tokencount'] = [None] * len(df.index)

def count_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

for i,data in enumerate(response.data):
    df["embeddings"][i] = data.embedding
    df["tokencount"][i] = count_tokens(df["Trends"][i])

/var/folders/fl/4w3n2t8d5m931spsglqv95d00000gn/T/ipykernel_40581/4057335757.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["embeddings"][i] = data.embedding
/var/folders/fl/4w3n2t8d5m931spsglqv95d00000gn/T/ipykernel_40581/4057335757

In [4]:
df

,URL,Trends,Source,embeddings,tokencount
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.020833317190408707, -0.022008126601576805,...",81
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.001784870750270784, -0.02892744168639183, ...",83
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.010437785647809505, -0.019189883023500443,...",119
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.015542690642178059, -0.005413859151303768,...",105
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.0049880193546414375, 0.001784318359568715,...",124
...,...,...,...,...,...
77,https://www.whowhatwear.com/spring-summer-2023...,"If lime green isn't your vibe, rest assured th...",Spring/Summer 2023 Fashion Trends: 21 Expert-A...,"[-0.002829852979630232, -0.01826709508895874, ...",100
78,https://www.whowhatwear.com/spring-summer-2023...,"""As someone who can clearly (not fondly) remem...",Spring/Summer 2023 Fashion Trends: 21 Expert-A...,"[-0.014651348814368248, -0.006454861257225275,...",100
79,https://www.whowhatwear.com/spring-summer-2023...,"""Combine this design shift with the fact that ...",Spring/Summer 2023 Fashion Trends: 21 Expert-A...,"[-0.020779484882950783, -0.025107983499765396,...",129
80,https://www.whowhatwear.com/spring-summer-2023...,Thought party season ended at the stroke of mi...,Spring/Summer 2023 Fashion Trends: 21 Expert-A...,"[-0.019850660115480423, -0.022311927750706673,...",105


In [5]:
df.to_csv('2023_fashion_trends_with_embeddings_and_Tokens.csv', index=False)

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [6]:
#Now that we have the embeddings and token count in Data, Lets create functions One to get the answer with or without context 
def get_answer(question,context=None):
   role = {"role": "system", "content":"You are a helpful assistant"}
   question = question if context is None else f"Given the context: {context} \nQuestion: {question}"
   gptquestion = {"role": "user", "content": question}
   messages = [role, gptquestion]
   response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
    )
   return response.choices[0].message.content

In [7]:
from embeddings_utils import distances_from_embeddings
def get_context(question):
    Query = openai.embeddings.create(input = question, model=EMBEDDING_MODEL_NAME)
    Query_embeddings = np.array(Query.data[0].embedding).flatten()
    df["embeddings"] = df["embeddings"].apply(lambda x: np.array(x).flatten())
    distances = distances_from_embeddings(Query_embeddings,df["embeddings"],distance_metric="cosine")
    df['distances'] = distances
    df.sort_values(by="distances", ascending=True)
    Context = "Context"
    token_count = 0
    i=0
    while token_count < 3000:
        Context = Context + "\n" + df["Trends"]
        token_count = token_count + df["tokencount"][i]
        i=i+1
    return Context


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [8]:
Question = "What are the fashion trends for 2023?"  
answer= get_answer(Question)
print("Regular answer from LLM, without Context is \n" + answer)
answer= get_answer(Question,get_context(Question))
print("\n\n\nContextual Answer is \n" + answer)

Regular answer from LLM, without Context is 
As fashion trends can change rapidly, it's hard to predict exactly what will be popular in 2023. However, based on current trends and industry forecasts, some possible fashion trends for 2023 could include:

1. Sustainability and eco-friendly fashion: With a growing awareness of environmental issues, sustainable and ethical fashion is expected to continue to gain popularity in 2023.

2. Oversized outerwear: Oversized jackets, coats, and blazers have been trending in recent years and are expected to continue in 2023.

3. Bright and bold colors: Vibrant colors and bold color combinations are expected to be popular in 2023, adding a touch of fun and excitement to outfits.

4. Vintage and retro styles: Nostalgic fashion trends from the 70s, 80s, and 90s are expected to make a comeback in 2023, giving a fresh twist to classic looks.

5. Tech-infused fashion: With the rise of smart clothing and wearable technology, we can expect to see more tech-i

### Question 2

In [9]:
Question = "What are the styles in Denims in 2023"  
answer= get_answer(Question)
print("Regular answer from LLM, without Context is \n" + answer)
answer= get_answer(Question,get_context(Question))
print("\n\n\nContextual Answer is \n" + answer)

Regular answer from LLM, without Context is 
In 2023, there are several denim styles that are popular and trending. Some of the key styles include:

1. Wide-leg jeans: Wide-leg jeans are making a comeback and are a popular choice for a relaxed and comfortable fit.

2. Straight-leg jeans: Straight-leg jeans continue to be a classic and versatile option that can be dressed up or down.

3. High-rise jeans: High-rise jeans remain a popular choice for a flattering and comfortable fit, with many different variations available.

4. Distressed jeans: Distressed denim, with rips, frays, and other worn-in details, continues to be a popular choice for a casual and edgy look.

5. Flare jeans: Flare jeans are also making a comeback in 2023, offering a retro-inspired and flattering silhouette.

6. Cropped jeans: Cropped jeans are a stylish option for showcasing your footwear and are a popular choice for a summery and casual look.

These are just a few of the denim styles that are trending in 2023. U

Conclusion: ChatGPT is giving Contextual answers from our Data. 